In [14]:
def plot_system(system,npl,mc,periods,m,mlow=np.array([]),mhi=np.array([]),rpl=np.array([]),planetname=np.array([]),plow=np.array([]), phi=np.array([])):
    #################################
    # check arguments
    ######################################

    if(npl <2):
        print('Need more than 1 planet')
        return
    if(np.size(rpl) == 0):
        rpl = np.zeros(npl)
        rpl = [1. for i in range (npl)]
    if(np.size(mlow)==0):
        mplow=m
    if(np.size(mhi) == 0):
        mphi = m
    if(np.size(planetname)==0):
        planetname = [i for i in range (npl)]
    
    #plot colors associated with each planet
    colors = ['blue','green','grey','goldenrod','sienna', 'red']


    mpl = [m[i]*3e-6 for i in range (npl)]
    mplow = [mlow[i]*3e-6 for i in range (npl)]
    mphi = [mhi[i]*3e-6 for i in range (npl)]
    
    #####################
    
    #make sure all the planets are sorted by period:
    idx   = np.argsort(periods)
    planetname = [planetname[i] for i in idx]
    mpl = np.array(mpl)[idx]
    mphi = np.array(mphi)[idx]
    mplow = np.array(mplow)[idx]
    periods = np.array(periods)[idx]
    rpl = np.array(rpl)[idx]

    #define the relevant eccentricity ranges
    apl = np.zeros(npl)
    apl = [ np.power( (mc*(periods[i]/365.25)*(periods[i]/365.25)),(0.3333333)) for i in range (npl)]
    elim = np.zeros(npl)
    for i in range (npl):
        if(i==0):
            elim[i] = apl[i+1]/apl[i] - 1.
        elif(i==npl-1):
            elim[i] = 1.-apl[i-1]/apl[i]
        else:
            e1 = apl[i+1]/apl[i] - 1.
            e2 = 1.-apl[i-1]/apl[i]
            if(e1 < e2):
                elim[i] = e1
            else:
                elim[i] = e2
    emax = np.amax(elim)
    emax =round(emax+0.02,2)
    ec = np.arange(0.02,emax,0.01)
    nec = len(ec)
    
    #define the set of resonances to be considered
    pq = [2,3,4,5,6,7,8,  3,5,7,9,    4,5,7,  5,7,9]
    q = [1,2,3,4,5,6,7,  1,3,5,7,    1,2,4,  1,3,5]
    nres = 17

    #have to define the array of axes
    junkfig, axs = plt.subplots(npl, 1, figsize=(0.1,0.1), sharex=True)
    
    
    
    #now set up the real figure
    fig = plt.figure(figsize=(4, 12))
    for i in range(npl):
        axs[i]=plt.subplot2grid((npl,1),(i,0),rowspan=1)
    for i in range(npl-1):
        axs[i].axes.get_xaxis().set_visible(False)

    plt.subplots_adjust(left=0.25, bottom=0.06, right=0.85, top=0.96, wspace=None, hspace=0.12)

    fig.suptitle(system)

    if(np.size(plow)==0 or np.size(phi)==0 ):
        #figure out the closest resonances to each planet to set the y-scaling
        allresperiods = np.zeros(2*npl*nres)
        j=0
        for pl in range(npl):
            for i in range(nres):
                allresperiods[j] = (float(q[i])/float(pq[i]))*periods[pl]
                j+=1
                allresperiods[j] = (float(pq[i])/float(q[i]))*periods[pl]
                j+=1

        allresperiods = np.sort(allresperiods)
        plow = np.zeros(npl)
        phi = np.zeros(npl)
        for pl in range(npl):
            absdiff = np.abs(allresperiods - periods[pl])
            closest = absdiff.argmin()
            if(allresperiods[closest] > periods[pl]):
                plow[pl] = 0.995*allresperiods[closest-1]
                phi[pl] = 1.005*allresperiods[closest] 
            else:
                plow[pl] = 0.995*allresperiods[closest]
                phi[pl] = 1.005*allresperiods[closest+1] 

            
            
    #make the plot
    for fpl in range(npl):
        ppl = periods[fpl] 
        ylow = plow[fpl] 
        yhigh =phi[fpl] 
    
        yd = yhigh-ylow
        yplabel = ppl - 0.1*yd
    
    
        ax = axs[npl-fpl-1]
        ax.axis([ 0, emax+0.02, ylow, yhigh])
        ax.set_frame_on(False)

        #redraw y axis
        ax.axvline(x=0.,color='k',linewidth=1)

        #draw a line at the planet's observed period
        ax.hlines(ppl,0,elim[fpl],color=colors[fpl])
        #draw a line to mark the eccentricity where it crosses its neighbor's orbit
        frac = 0.05*yd
        ax.vlines(elim[fpl],ppl-frac,ppl+frac,color = colors[fpl],linestyle = "-")

    
        ax.text(0.01,yplabel,planetname[fpl],color=colors[fpl])

        #add a marker scaled with planet radius
        ax.scatter((0.01),(ppl),marker="o",color=colors[fpl], s=30.*rpl[fpl]**2.)
        for pl in range(npl):
            if(pl == fpl):
                continue
            for i in range(nres):

                #semimajor axis ratio
                alpha = np.power((float(q[i])/float(pq[i])),(2./3.))


                #interior resonances
                ipres = (float(q[i])/float(pq[i]))*periods[pl]

                #check if the resonance falls in the plot range:
                if( (1.025*ipres) > ylow and (0.975*ipres) < yhigh ):
                    order = pq[i] - q[i]
                    fd = fd_int(pq[i],q[i])
                    if(order >1):
                        #lower mass limit
                        Cr = np.abs((mplow[pl]/mc)*alpha*fd)
                        dx1 = [( np.sqrt(12.*Cr*np.power(ec[k],float(order))) ) for k in range(nec) ]
                        #upper masss limit
                        Cr = np.abs((mphi[pl]/mc)*alpha*fd)                    
                        dx2 = [( np.sqrt(12.*Cr*np.power(ec[k],float(order))) ) for k in range(nec) ]
                    if(order == 1):
                        fd = fd_intq1(pq[i],alpha)
                        #lower mass limit
                        Cr = np.abs((mplow[pl]/mc)*alpha*fd)
                        dx1 = np.zeros(nec)
                        j = float(pq[i])
                        j2 = 1.-j
                        for k in range(nec):
                            t1 = np.sqrt(12.*Cr*ec[k])
                            t2 = np.sqrt(1. + Cr/(27.*j2*j2*ec[k]*ec[k]*ec[k]))
                            t3 = -Cr/(3.*j2*ec[k])
                            dx1[k] = t1*t2 + t3
                        #upper masss limit
                        #calculate Cr/n because the n gets removed anyway
                        Cr = np.abs((mphi[pl]/mc)*alpha*fd)
                        dx2 = np.zeros(nec)
                        for k in range(nec):
                            t1 = np.sqrt(12.*Cr*ec[k])
                            t2 = np.sqrt(1. + Cr/(27.*j2*j2*ec[k]*ec[k]*ec[k]))
                            t3 = -Cr/(3.*j2*ec[k])
                            dx2[k] = t1*t2 + t3
                    xin1 = [ ipres*(1. - dx1[k]) for k in range (nec)]
                    xout1 = [ ipres*(1. + dx1[k]) for k in range (nec)]

                    xin2 = [  ipres*(1. - dx2[k]) for k in range (nec)]
                    xout2 = [  ipres*(1. + dx2[k]) for k in range (nec)]                        

                    xmin = np.amin(xin2)
                    xmax = np.amin(xout2)
                    if( xmax > ylow and xmin < yhigh ):
                        label = str(pq[i]) + ':' + str(q[i]) + '-' + planets[pl]
                        ax.text(emax+0.02,ipres,label,color=colors[pl],verticalalignment='center')
                        x = [ipres for k in range (nec)]
                        ax.plot(ec,x,color=colors[pl],linestyle=':',linewidth=1)

                        ax.fill_between(ec,xin1,xout1,facecolor=colors[pl],alpha = 0.4)
                        ax.fill_between(ec,xin1,xin2,facecolor=colors[pl],alpha = 0.2)
                        ax.fill_between(ec,xout1,xout2,facecolor=colors[pl],alpha = 0.2)



                #exterior resonance
                epres = (float(pq[i])/float(q[i]))*periods[pl]
                if( (1.025*epres) > ylow and (0.975*epres) < yhigh ):
                    order = pq[i] - q[i]
                    fd = fd_ext(pq[i],q[i])
                    if(order >1):                       
                        #lower mass limit
                        Cr = np.abs((mplow[pl]/mc)*fd)
                        dx1 = [( np.sqrt(12.*Cr*np.power(ec[k],float(order))) ) for k in range(nec) ]
                        #upper masss limit
                        Cr = np.abs((mphi[pl]/mc)*fd)                    
                        dx2 = [( np.sqrt(12.*Cr*np.power(ec[k],float(order))) ) for k in range(nec) ]              
                    if(order == 1):
                        #lower mass limit
                        Cr = np.abs((mplow[pl]/mc)*fd)
                        dx1 = np.zeros(nec)
                        j = float(-q[i])
                        j2 = 1.-j
                        for k in range(nec):
                            t1 = np.sqrt(12.*Cr*ec[k])
                            t2 = np.sqrt(1. + Cr/(27.*j2*j2*ec[k]*ec[k]*ec[k]))
                            t3 = -Cr/(3.*j2*ec[k])
                            dx1[k] = t1*t2 + t3
                        #upper masss limit
                        Cr = np.abs((mphi[pl]/mc)*fd)
                        dx2 = np.zeros(nec)
                        for k in range(nec):
                            t1 = np.sqrt(12.*Cr*ec[k])
                            t2 = np.sqrt(1. + Cr/(27.*j2*j2*ec[k]*ec[k]*ec[k]))
                            t3 = -Cr/(3.*j2*ec[k])
                            dx2[k] = t1*t2 + t3

                    xin1 = [ epres*(1. - dx1[k]) for k in range (nec)]
                    xout1 = [ epres*(1. + dx1[k]) for k in range (nec)]


                    xin2 = [  epres*(1. - dx2[k]) for k in range (nec)]
                    xout2 = [  epres*(1. + dx2[k]) for k in range (nec)]

                    xmin = np.amin(xin2)
                    xmax = np.amin(xout2)
                    if( xmax > ylow and xmin < yhigh ):
                        label = str(pq[i]) + '-' + planets[pl] + ':' + str(q[i])

                        ax.text(emax+0.02,epres,label,color=colors[pl],verticalalignment='center')
                        x = [epres for k in range (nec)]
                        ax.plot(ec,x,color=colors[pl],linestyle=':',linewidth=1)

                        ax.fill_between(ec,xin1,xout1,facecolor=colors[pl],alpha = 0.4)
                        ax.fill_between(ec,xin1,xin2,facecolor=colors[pl],alpha = 0.2)
                        ax.fill_between(ec,xout1,xout2,facecolor=colors[pl],alpha = 0.2)

    d = .015 
    kwargs = dict(transform=axs[0].transAxes, color='k', clip_on=False)
    axs[0].plot((-d, +d), (-d, +d), **kwargs)    
    for i in range(1,npl-1):
        kwargs.update(transform=axs[i].transAxes)  
        axs[i].plot((-d, +d), (-d, +d), **kwargs)
        axs[i].plot((-d, +d), (1 - d, 1 + d), **kwargs)   
    kwargs.update(transform=axs[npl-1].transAxes)  
    axs[i].plot((-d, +d), (1 - d, 1 + d), **kwargs)   

    axs[npl-1].axhline(y=plow[0],color='k',linewidth=1.5) 


    fig.text(0.595, 0.015, 'eccentricity', ha='center',fontsize=16)
    fig.text(0.035, 0.5, 'orbital period (days)', va='center', rotation='vertical',fontsize=16)            

            
    return     
            
            
            
            
            
            